In [50]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils

In [51]:
#setting random seed for the purpose of reproducability
seed = 21

In [52]:
#importing the data
from keras.datasets import cifar10

In [53]:
#loading the data
(X_train,y_train), (X_test, y_test) = cifar10.load_data()

In [54]:
print(X_train[0])

[[[ 59  62  63]
  [ 43  46  45]
  [ 50  48  43]
  ...
  [158 132 108]
  [152 125 102]
  [148 124 103]]

 [[ 16  20  20]
  [  0   0   0]
  [ 18   8   0]
  ...
  [123  88  55]
  [119  83  50]
  [122  87  57]]

 [[ 25  24  21]
  [ 16   7   0]
  [ 49  27   8]
  ...
  [118  84  50]
  [120  84  50]
  [109  73  42]]

 ...

 [[208 170  96]
  [201 153  34]
  [198 161  26]
  ...
  [160 133  70]
  [ 56  31   7]
  [ 53  34  20]]

 [[180 139  96]
  [173 123  42]
  [186 144  30]
  ...
  [184 148  94]
  [ 97  62  34]
  [ 83  53  34]]

 [[177 144 116]
  [168 129  94]
  [179 142  87]
  ...
  [216 184 140]
  [151 118  84]
  [123  92  72]]]


In [55]:
#normalize the inputs from 0-255 to between 0 and 1 by dividing by 255


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train/255.0
X_test = X_test/255.0

In [56]:
#one hot encoding outputs

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]

In [57]:
#designing the model

model = Sequential()

In [58]:
#convolution layer
#zero padding done

model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))

In [59]:
model.add(Dropout(0.2))

In [60]:
model.add(BatchNormalization())

In [61]:
#another convolution layer 
#filter size increases so that network can learn more complex representations

model.add(Conv2D(64, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))

In [62]:
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [63]:
#repeating the same

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [64]:
model.add(Flatten())
model.add(Dropout(0.2))


In [65]:
model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [66]:
model.add(Dense(class_num))
model.add(Activation('softmax'))

In [67]:
epochs = 25
optimizer = 'adam'

In [68]:
#compile the model with chosen parameter, specify metrics
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics = ['accuracy'])

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics = ['accuracy])

In [69]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 activation_11 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 dropout_12 (Dropout)        (None, 32, 32, 32)        0         
                                                                 
 batch_normalization_4 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 conv2d_14 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 activation_12 (Activation)  (None, 32, 32, 64)        0         
                                                      

In [70]:
numpy.random.seed(seed)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)

Epoch 1/25
782/782 [==============================] - 191s 243ms/step - loss: 1.5062 - accuracy: 0.4656 - val_loss: 1.1704 - val_accuracy: 0.5805
Epoch 2/25
782/782 [==============================] - 212s 271ms/step - loss: 1.0299 - accuracy: 0.6351 - val_loss: 0.8353 - val_accuracy: 0.7081
Epoch 3/25
782/782 [==============================] - 181s 231ms/step - loss: 0.8484 - accuracy: 0.7018 - val_loss: 0.8030 - val_accuracy: 0.7239
Epoch 4/25
782/782 [==============================] - 127s 163ms/step - loss: 0.7591 - accuracy: 0.7326 - val_loss: 0.7234 - val_accuracy: 0.7544
Epoch 5/25
782/782 [==============================] - 182s 233ms/step - loss: 0.6980 - accuracy: 0.7576 - val_loss: 0.6347 - val_accuracy: 0.7808
Epoch 6/25
782/782 [==============================] - 185s 237ms/step - loss: 0.6577 - accuracy: 0.7690 - val_loss: 0.6756 - val_accuracy: 0.7679
Epoch 7/25
782/782 [==============================] - 210s 269ms/step - loss: 0.6226 - accuracy: 0.7820 - val_loss: 0.6534 -

In [71]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 82.01%
